In [1]:
!nvidia-smi

Tue Aug 26 12:59:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   43C    P8             11W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
%%shell
# Adapted from the official dockerfile: https://github.com/google-deepmind/alphafold3/blob/main/docker/Dockerfile
rm -rf /content/sample_data

# /app/alphafold - alphafold3 git repo checkout
mkdir /app
cd /app
git clone https://github.com/google-deepmind/alphafold3.git alphafold
apt-get install --yes --quiet python3.12 python3-pip python3.12-venv python3.12-dev git wget gcc g++ make zlib1g-dev zstd

# /alphafold3_venv - pip environment for alphafold3 dependancies
python3 -m venv /alphafold3_venv
source /alphafold3_venv/bin/activate

# /hmmer - build hmmer from source & custom patch
mkdir /hmmer_build /hmmer
wget http://eddylab.org/software/hmmer/hmmer-3.4.tar.gz --directory-prefix /hmmer_build
cd /hmmer_build && echo "ca70d94fd0cf271bd7063423aabb116d42de533117343a9b27a65c17ff06fbf3 hmmer-3.4.tar.gz" | sha256sum --check
cd /hmmer_build && tar zxf hmmer-3.4.tar.gz && rm hmmer-3.4.tar.gz

cp /app/alphafold/docker/jackhmmer_seq_limit.patch /hmmer_build/
cd /hmmer_build && patch -p0 < jackhmmer_seq_limit.patch

cd /hmmer_build/hmmer-3.4 && ./configure --prefix /hmmer
cd /hmmer_build/hmmer-3.4 && make -j
cd /hmmer_build/hmmer-3.4 && make install
cd /hmmer_build/hmmer-3.4/easel && make install
rm -R /hmmer_build

# build pip dependencies & ligand dictionary
cd /app/alphafold
pip3 install --no-cache-dir setuptools
pip3 install --no-cache-dir -r dev-requirements.txt
pip3 install --no-cache-dir --no-deps .
build_data

Cloning into 'alphafold'...
remote: Enumerating objects: 1443, done.
remote: Counting objects: 100% (897/897), done.
remote: Compressing objects: 100% (401/401), done.
remote: Total 1443 (delta 648), reused 496 (delta 496), pack-reused 546 (from 3)
Receiving objects: 100% (1443/1443), 40.71 MiB | 16.81 MiB/s, done.
Resolving deltas: 100% (851/851), done.
Reading package lists...
Building dependency tree...
Reading state information...
g++ is already the newest version (4:11.2.0-1ubuntu1).
g++ set to manually installed.
gcc is already the newest version (4:11.2.0-1ubuntu1).
gcc set to manually installed.
make is already the newest version (4.3-4.1build1).
make set to manually installed.
git is already the newest version (1:2.34.1-1ubuntu1.15).
wget is already the newest version (1.21.2-2ubuntu1.1).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-2ubuntu9.2).
zlib1g-dev set to manually installed.
python3.12 is already the newest version (3.12.11-1+jammy1).
python3.12-dev is alrea

In [3]:
# Copy weights from Google Drive to the default location at /root/models/af3.bin.zst
from google.colab import drive
drive.mount('/content/drive')
!mkdir -p /root/models/
!cp /content/drive/MyDrive/alphafold3/models/af3.bin.zst /root/models/af3.bin.zst
!md5sum /root/models/af3.bin.zst

Mounted at /content/drive
ea076e2d05168d66a3d9a3fa62e029d0  /root/models/af3.bin.zst


In [4]:
%%shell
source /alphafold3_venv/bin/activate
PATH="/hmmer/bin:$PATH"
cd /app/alphafold
XLA_PYTHON_CLIENT_PREALLOCATE=true
XLA_CLIENT_MEM_FRACTION=0.95
python3 run_alphafold_data_test.py

Running tests under Python 3.12.11: /alphafold3_venv/bin/python3
[ RUN      ] DataPipelineTest.test_config
[       OK ] DataPipelineTest.test_config
[ RUN      ] DataPipelineTest.test_featurisation
Running data pipeline for chain P...
I0826 13:07:12.108162 138818826556032 pipeline.py:82] Getting protein MSAs for sequence SEFEKLRQTGDELVQAFQRLREIFDKGDDDSLEQVLEEIEELIQKHRQLFDNRQEAADTEAAKQGDQWVQLFQRFREAIDKGDKDSLEQLLEELEQALQKIRELAEKKN
I0826 13:07:12.110995 138817275369024 jackhmmer.py:85] Query sequence: SEFEKLRQTGDELVQA... (len 109)
I0826 13:07:12.111133 138817266976320 jackhmmer.py:85] Query sequence: SEFEKLRQTGDELVQA... (len 109)
I0826 13:07:12.111634 138817250190912 jackhmmer.py:85] Query sequence: SEFEKLRQTGDELVQA... (len 109)
I0826 13:07:12.111737 138817258583616 jackhmmer.py:85] Query sequence: SEFEKLRQTGDELVQA... (len 109)
I0826 13:07:12.111828 138817275369024 subprocess_utils.py:82] Launching subprocess "/hmmer/bin/jackhmmer -o /dev/null -A /tmp/tmpfeqwixx7/output.sto --noali --F1 0

In [5]:
%%shell
source /alphafold3_venv/bin/activate
PATH="/hmmer/bin:$PATH"
cd /app/alphafold
XLA_PYTHON_CLIENT_PREALLOCATE=true
XLA_CLIENT_MEM_FRACTION=0.95
python3 run_alphafold_test.py

Running tests under Python 3.12.11: /alphafold3_venv/bin/python3
[ RUN      ] InferenceTest.test_inference_bucket_1024
I0826 13:07:24.146206 133198563570304 xla_bridge.py:895] Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
I0826 13:07:24.147384 133198563570304 xla_bridge.py:895] Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory

Running fold job 5tgy...
Output will be written in /tmp/absl_testing/InferenceTest/test_inference_bucket_1024/tmpgkg6th59
Running data pipeline...
Running data pipeline for chain P...
I0826 13:07:24.149144 133198563570304 pipeline.py:82] Getting protein MSAs for sequence SEFEKLRQTGDELVQAFQRLREIFDKGDDDSLEQVLEEIEELIQKHRQLFDNRQEAADTEAAKQGDQWVQLFQRFREAIDKGDKDSLEQLLEELEQALQKIRELAEKKN
I0826 13:07:24.152223 133193042753088 jackhmmer.py:85] Query sequence: SEFEKLRQTGDELVQA... (len 109)
I0826 13:07:24.152354 133193025967680